In [1]:
import numpy as np
import torch
import pyepo

Auto-Sklearn cannot be imported.


## Data Set and Optimization Solver

In [2]:
# generate data
num_node = 50 # node size
num_data = 100 # number of training data
num_feat = 10 # size of feature
deg = 4 # polynomial degree
e = 0.5 # noise width
feats, costs = pyepo.data.tsp.genData(num_data, num_feat, num_node, deg, e, seed=42)

In [3]:
from model import tspDFJModel
# set solver
optmodel = tspDFJModel(num_node)
# test
optmodel.setObj(costs[0])
sol, obj = optmodel.solve()
print("Obj: {}".format(obj))
tour = optmodel.getTour(sol)
print(" -> ".join(map(str, tour)))

Set parameter Username
Academic license - for non-commercial use only - expires 2024-01-01
Obj: 81.7767
0 -> 16 -> 37 -> 32 -> 34 -> 17 -> 40 -> 41 -> 12 -> 1 -> 9 -> 22 -> 23 -> 20 -> 26 -> 43 -> 19 -> 6 -> 14 -> 44 -> 24 -> 48 -> 29 -> 15 -> 10 -> 30 -> 47 -> 11 -> 27 -> 2 -> 36 -> 18 -> 7 -> 13 -> 8 -> 49 -> 33 -> 35 -> 4 -> 25 -> 46 -> 39 -> 45 -> 21 -> 42 -> 5 -> 3 -> 31 -> 28 -> 38 -> 0


In [4]:
from dataset import optDatasetConstrs
dataset_train = optDatasetConstrs(optmodel, feats, costs=costs) # with binding constr

Optimizing for optDataset...


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:19<00:00,  5.09it/s]


In [5]:
# get data loader
from torch.utils.data import DataLoader
from dataset import collate_fn
batch_size = 32
loader_train = DataLoader(dataset_train, batch_size=batch_size, collate_fn=collate_fn, shuffle=True)

## Prediction Model

In [6]:
import torch
from torch import nn

# build linear model
class LinearRegression(nn.Module):

    def __init__(self):
        super(LinearRegression, self).__init__()
        self.linear = nn.Linear(num_feat, num_node*(num_node-1)//2)

    def forward(self, x):
        out = self.linear(x)
        return out

## Train

In [7]:
lr = 5e-2
num_epochs = 1

In [8]:
import time
from tqdm import tqdm

def pipeline(reg, forward_func, loss_func, lr, num_epochs, loader_train, seed=42):
    # set random seed
    np.random.seed(42)
    torch.manual_seed(42)
    # set optimizer
    optimizer = torch.optim.Adam(reg.parameters(), lr=lr)
    # running time
    elapsed = 0
    tbar = tqdm(range(num_epochs))
    for epoch in tbar:
        # training
        time.sleep(1)
        tick = time.time()
        for data in loader_train:
            # forward pass)
            loss = forward_func(data, reg, loss_func)
            # backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            tbar.set_description("Loss: {:8.4f}".format(loss.item()))
        # record time
        tock = time.time()
        elapsed += tock - tick
    time.sleep(1)
    print("Evaluation:")
    print("Training Elapsed Time: {:.2f} Sec".format(elapsed))

def forwardCAVE(data, reg, loss_func):
    # unzip data
    x, _, _, t_ctr = data
    # predict
    cp = reg(x)
    # loss
    loss = loss_func(cp, t_ctr)
    return loss

### Standard

In [9]:
# init model
reg = LinearRegression()

In [10]:
from func import exactConeAlignedCosine
# init loss
ca_cos = exactConeAlignedCosine(optmodel, warmstart=False, conecheck=False, processes=1)

Num of cores: 1


In [11]:
pipeline(reg, forwardCAVE, ca_cos, lr, num_epochs, loader_train)

Loss:  -0.9737: 100%|████████████████████████████████████████████████████████████████████| 1/1 [00:23<00:00, 23.20s/it]


Evaluation:
Training Elapsed Time: 22.19 Sec


### Multi-Core 

In [12]:
# init model
reg = LinearRegression()

In [13]:
from func import exactConeAlignedCosine
# init loss
ca_cos = exactConeAlignedCosine(optmodel, warmstart=False, conecheck=False, processes=4)

Num of cores: 4


In [14]:
pipeline(reg, forwardCAVE, ca_cos, lr, num_epochs, loader_train)

Loss:  -0.9717: 100%|████████████████████████████████████████████████████████████████████| 1/1 [00:17<00:00, 17.09s/it]


Evaluation:
Training Elapsed Time: 16.08 Sec


### Warm Starting 

In [15]:
# init model
reg = LinearRegression()

In [16]:
from func import exactConeAlignedCosine
# init loss
ca_cos = exactConeAlignedCosine(optmodel, warmstart=True, conecheck=False, processes=1)

Num of cores: 1


In [17]:
pipeline(reg, forwardCAVE, ca_cos, lr, num_epochs, loader_train)

Loss:  -0.9717: 100%|████████████████████████████████████████████████████████████████████| 1/1 [00:22<00:00, 22.57s/it]


Evaluation:
Training Elapsed Time: 21.56 Sec


### Cone Check 

In [18]:
# init model
reg = LinearRegression()

In [19]:
from func import exactConeAlignedCosine
# init loss
ca_cos = exactConeAlignedCosine(optmodel, warmstart=False, conecheck=True, processes=1)

Num of cores: 1


In [20]:
pipeline(reg, forwardCAVE, ca_cos, lr, num_epochs, loader_train)

Loss:  -0.9717: 100%|████████████████████████████████████████████████████████████████████| 1/1 [00:23<00:00, 23.91s/it]


Evaluation:
Training Elapsed Time: 22.90 Sec
